### Some practical applications of audio classification include identifying speaker intent, language classification, and even animal species by their sound

In [2]:
# !python -m pip install -U -q transformers datasets evaluate soundfile ipywidgets librosa

In [ ]:
# !python -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
# !python -m pip install accelerate -U

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from huggingface_hub import whoami

whoami()
# you should see something like {'type': 'user',  'id': '...',  'name': 'Wauplin', ...}

{'type': 'user',
 'id': '658f9e91b72f4ce63b2304b2',
 'name': 'stevehoang9',
 'fullname': 'Phạm Văn Hoàng',
 'email': 'hoang.pv192875@sis.hust.edu.vn',
 'emailVerified': True,
 'canPay': False,
 'periodEnd': None,
 'isPro': False,
 'avatarUrl': '/avatars/fd1415c77796196c6ee2b923d73df0eb.svg',
 'orgs': [],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'HFToken', 'role': 'write'}}}

In [3]:
from datasets import load_dataset, Audio 

minds = load_dataset("PolyAI/minds14", name="en-US", split="train")

/media/sanslab/Data/stevehoang/SpeechToCommand/venv/lib/python3.11/site-packages/datasets/load.py:1486: FutureWarning: The repository for PolyAI/minds14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/minds14
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [4]:
minds = minds.train_test_split(test_size=0.2)

In [5]:
minds

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 450
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 113
    })
})

In [6]:
minds = minds.remove_columns(["path", "transcription", "english_transcription", "lang_id"])

In [7]:
minds["train"][0]

{'audio': {'path': '/home/sanslab/.cache/huggingface/datasets/downloads/extracted/e573c870e54f06ccbe110214dbc51c75280a2a7a2d21d69704b9634577fad316/en-US~HIGH_VALUE_PAYMENT/602b9d30bb1e6d0fbce91fa1.wav',
  'array': array([0.00024414, 0.        , 0.00024414, ..., 0.        , 0.        ,
         0.00024414]),
  'sampling_rate': 8000},
 'intent_class': 10}

In [8]:
labels = minds["train"].features["intent_class"].names
labels

['abroad',
 'address',
 'app_error',
 'atm_limit',
 'balance',
 'business_loan',
 'card_issues',
 'cash_deposit',
 'direct_debit',
 'freeze',
 'high_value_payment',
 'joint_account',
 'latest_transactions',
 'pay_bill']

In [9]:
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label 

In [10]:
id2label[str(2)]

'app_error'

In [11]:
label2id['app_error']

'2'

In [12]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/media/sanslab/Data/stevehoang/SpeechToCommand/venv/lib/python3.11/site-packages/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [13]:
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))
minds["train"][0]

{'audio': {'path': '/home/sanslab/.cache/huggingface/datasets/downloads/extracted/e573c870e54f06ccbe110214dbc51c75280a2a7a2d21d69704b9634577fad316/en-US~HIGH_VALUE_PAYMENT/602b9d30bb1e6d0fbce91fa1.wav',
  'array': array([2.16851447e-04, 9.96214803e-05, 2.74515623e-05, ...,
         1.98202761e-04, 2.64456437e-04, 1.23714417e-04]),
  'sampling_rate': 16000},
 'intent_class': 10}

In [14]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs 

In [15]:
encoded_minds = minds.map(preprocess_function, remove_columns="audio", batched=True)
encoded_minds = encoded_minds.rename_column("intent_class", "label")

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

In [16]:
encoded_minds

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values'],
        num_rows: 450
    })
    test: Dataset({
        features: ['label', 'input_values'],
        num_rows: 113
    })
})

In [17]:
import evaluate 

accuracy = evaluate.load("accuracy")

In [18]:
import numpy as np 

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [19]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

/media/sanslab/Data/stevehoang/SpeechToCommand/venv/lib/python3.11/site-packages/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
training_args = TrainingArguments(
    output_dir="my_awesome_mind_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,No log,2.645326,0.079646
1,No log,2.650372,0.035398
2,2.636600,2.650723,0.053097
4,2.636600,2.658453,0.061947
5,2.627600,2.663316,0.053097
6,2.627600,2.665159,0.044248
8,2.616400,2.665313,0.035398


TrainOutput(global_step=30, training_loss=2.626864751180013, metrics={'train_runtime': 35.0625, 'train_samples_per_second': 128.342, 'train_steps_per_second': 0.856, 'total_flos': 3.26841433344e+16, 'train_loss': 2.626864751180013, 'epoch': 8.0})

In [22]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/stevehoang9/my_awesome_mind_model/commit/62aaf340696b88a8b4e86cf5ee10deee5b0cd19a', commit_message='End of training', commit_description='', oid='62aaf340696b88a8b4e86cf5ee10deee5b0cd19a', pr_url=None, pr_revision=None, pr_num=None)

In [23]:
from datasets import load_dataset, Audio 

dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
sampling_rate = dataset.features["audio"].sampling_rate 
audio_file = dataset[0]["audio"]["path"]

/media/sanslab/Data/stevehoang/SpeechToCommand/venv/lib/python3.11/site-packages/datasets/load.py:1486: FutureWarning: The repository for PolyAI/minds14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/minds14
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [25]:
from transformers import pipeline 

classifier = pipeline("audio-classification", model="stevehoang9/my_awesome_mind_model")
classifier(audio_file)

config.json:   0%|          | 0.00/3.10k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

[{'score': 0.07704444229602814, 'label': 'address'},
 {'score': 0.07559876143932343, 'label': 'joint_account'},
 {'score': 0.07297999411821365, 'label': 'pay_bill'},
 {'score': 0.07267731428146362, 'label': 'cash_deposit'},
 {'score': 0.07258542627096176, 'label': 'business_loan'}]

In [26]:
from transformers import AutoFeatureExtractor 

feature_extractor = AutoFeatureExtractor.from_pretrained("stevehoang9/my_awesome_mind_model")
inputs = feature_extractor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")

In [27]:
from transformers import AutoModelForAudioClassification
import torch

model = AutoModelForAudioClassification.from_pretrained("stevehoang9/my_awesome_mind_model")
with torch.no_grad():
    logits = model(**inputs).logits

In [29]:
predicted_class_ids = torch.argmax(logits).item()
predicted_label = model.config.id2label[predicted_class_ids]
predicted_label

'address'